In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import itertools
import obj # my .py file with classes for individual athletes and countries

pd.options.mode.chained_assignment = None

In [2]:
women = pd.read_csv('women.csv')

**Create a projected field of gymnasts for which we can estimate off of**

In [3]:
women_quals = ['USA', 'GBR', 'CAN', 'CHN', 'BRA', 'ITA', 'NED', 'FRA', 'JPN', 'AUS', 'ROU', 'KOR']

women['qual_team'] = women['Olympic_Nation'].isin(women_quals)

In [4]:
# Create a dictionary with the expected 5 gymnasts for each of the qualifying teams

w_comp_dict = {} # dictionary of key = country, value = dictionary of key = Gymnast, value = DataFrame of Average & SD Performance in each apparatus
no_USA = women_quals.copy()
no_USA.remove('USA')
for country in no_USA:
    w_comp_dict[country] = obj.w_country(country).top_5
    
q_ath = set()
for country in w_comp_dict:
    for ath in w_comp_dict[country]:
        q_ath.add(ath)

In [5]:
# Already Qualified Individuals from the 2023 World Championships: https://usagym.org/events/2023-artistic-gymnastics-world-championships/

w_App_qual = [
    ('Csenge BACSKAY', 'VT'),
    ('Ahtziri SANDOVAL', 'UB'),
    ('Ana PEREZ', 'BB'),
    ('Sarah VOSS', 'FX')]

w_AA_qual = ['Kaylia NEMOUR',
             'Alexa MORENO',
             'Filipa MARTINS',
             'Aleah FINNEGAN',
             'Bettina CZIFRA',
             'Alba PETISCO',
             'Anna LASHCHEVSKA',
             'Lena BICKEL',
             'Hillary HERON',
             'Caitlin ROOSKRANTZ',
             'Sona ARTAMONOVA',
             'Lihie RAZ',
             'Lucija HRIBAR',
             'Luisa BLANCO',
             'Rifda IRFANALUTHFI',
             'Lynnzee BROWN'] # manually chosen Tripartite selection, suggestion from https://www.reddit.com/r/Gymnastics/comments/175130d/tripartite_commission_invitations/

In [6]:
# Estimating who will be sent from the first 3 countries that did not make it as a team

w_countries_1 = ['GER', 'MEX', 'HUN']
idv_qualed = [a[0] for a in w_App_qual] + w_AA_qual
for country_1 in w_countries_1:
    country_aths = obj.w_country(country_1).ath_perf
    top_score = 0
    top_scorer = None
    for ath in country_aths:
        if len(country_aths[ath]) == 6 and np.mean(country_aths[ath]['mean']) > top_score and ath not in idv_qualed:
            top_score = np.mean(country_aths[ath]['mean'])
            top_scorer = ath
    w_AA_qual.append(top_scorer)

In [7]:
# update qual column so all individual qualifiers so far are included as True
idv_qualed = [a[0] for a in w_App_qual] + w_AA_qual
women.loc[women['Name'].isin(idv_qualed), 'qual_team'] = True 

In [8]:
remaining = women[women['qual_team'] == False]['Name'].unique()
last_4_AA = {}
tracking_AA_d = {}
for ath in remaining:
    ath_summ = obj.female_gymnast(ath).summ_table
    if len(ath_summ) == 6:
        if len(last_4_AA) < 4:
            last_4_AA[ath] = sum(ath_summ['mean'])
            tracking_AA_d[sum(ath_summ['mean'])] = ath
        elif sum(ath_summ['mean']) > min(last_4_AA.values()):
            last_4_AA[ath] = sum(ath_summ['mean'])
            tracking_AA_d[sum(ath_summ['mean'])] = ath
            last_4_AA.pop(tracking_AA_d[min(last_4_AA.values())])

w_AA_qual.extend(last_4_AA.keys())

In [9]:
last_2_App = {'FX': {}, 'BB': {}, 'UB': {}, 'VT': {}}

for ath in remaining:
    if ath not in last_4_AA:
        ath_summ = obj.female_gymnast(ath).summ_table
        if ath not in last_4_AA:
            for app in ath_summ.itertuples():
                last_2_App[app.Index][ath] = app.mean

In [10]:
for app in last_2_App:
    app_scores = last_2_App[app]
    top_aths = sorted(app_scores, key=lambda k: app_scores[k], reverse=True)[:2]
    for ath in top_aths:
        w_App_qual.append((ath, app))

In [11]:
for AA in w_AA_qual:
    a = obj.female_gymnast(AA)
    if a.country in w_comp_dict:
        w_comp_dict[a.country][AA] = a.summ_table
    else:
        w_comp_dict[a.country] = {AA: a.summ_table}

In [12]:
for App in w_App_qual:
    a = obj.female_gymnast(App[0])
    app_row = a.summ_table.loc[App[1]].to_frame().T
    if a.country in w_comp_dict:
        w_comp_dict[a.country][App[0]] = app_row
    else:
        w_comp_dict[a.country] = {App[0]: app_row}

In [13]:
qual = {'FX': [], 'BB': [], 'UB': [], 'VT': []}
score_range = np.arange(12, 17, 0.001)
for country in w_comp_dict:
    country_comp = {'FX': [], 'BB': [], 'UB': [], 'VT': []}
    country_mean_scores = {'FX': [], 'BB': [], 'UB': [], 'VT': []}
    for gymnast in w_comp_dict[country]:
        for apparatus in w_comp_dict[country][gymnast].itertuples():
            if len(country_comp[apparatus.Index]) < 4:
                dist = norm.cdf(score_range, loc=apparatus.mean, scale=apparatus.std_dev)
                country_comp[apparatus.Index].append((apparatus.mean, apparatus.std_dev, dist, gymnast, country))
                country_mean_scores[apparatus.Index].append(apparatus.mean)
           
            elif apparatus.mean > min(country_mean_scores[apparatus.Index]):
                dist = norm.cdf(score_range, loc=apparatus.mean, scale=apparatus.std_dev)
                country_comp[apparatus.Index].append((apparatus.mean, apparatus.std_dev, dist, gymnast, country))
                for i, p in enumerate(country_comp[apparatus.Index]):
                    if min(country_mean_scores[apparatus.Index]) == p[0]:
                        country_comp[apparatus.Index].pop(i)                        
    for apparatus in qual:
        qual[apparatus].extend(country_comp[apparatus])

C:\Users\lukel\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2176: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


In [14]:
women_apps = ['BB', 'FX', 'UB', 'VT']

n = 100

qual_scores = {}
athlete_dict = {}
for apparatus in women_apps: # Run Qualifying Rounds
    for athlete in qual[apparatus]:
        qual_scores[athlete[3] + '-' + athlete[4] + '_' + apparatus] = np.random.normal(athlete[0], athlete[1], n)
        athlete_dict[athlete[3] + '-' + athlete[4] +'_' + apparatus] = athlete
sims = pd.DataFrame(qual_scores)

In [21]:
sims.to_csv('w_sims.csv', index = False)

In [24]:
sims

,Alice KINSELLA-GBR_BB,Georgia FENTON-GBR_BB,Ondine ACHAMPONG-GBR_BB,Jessica GADIROVA-GBR_BB,Ellie BLACK-CAN_BB,Emma SPENCE-CAN_BB,Sydney TURNER-CAN_BB,Ava STEWART-CAN_BB,Ran WU-CHN_BB,Yaqin ZHOU-CHN_BB,...,Lena BICKEL-SUI_VT,Hillary HERON-PAN_VT,Caitlin ROOSKRANTZ-RSA_VT,Sona ARTAMONOVA-CZE_VT,Lihie RAZ-ISR_VT,Lucija HRIBAR-SLO_VT,Luisa BLANCO-COL_VT,Rifda IRFANALUTHFI-INA_VT,Lynnzee BROWN-HAI_VT,Changok AN-PRK_VT
0,12.485103,12.948466,14.007085,12.088857,13.055136,12.520513,12.938275,13.498204,14.633,14.604278,...,13.179275,10.921510,13.268795,12.410265,13.544751,13.363210,13.813320,13.104923,13.459378,13.818438
1,13.197856,13.101890,13.587160,13.653701,13.791839,12.473492,11.886694,12.155113,14.633,14.471585,...,13.107151,10.644948,12.785819,13.658336,12.787104,13.087998,13.666530,13.106891,13.464719,13.937586
2,13.149136,13.497935,12.731996,13.079219,13.108124,13.285027,13.532673,11.965466,14.633,14.389520,...,12.976814,11.631379,13.185154,13.185495,12.880022,13.065205,13.244929,13.046948,13.385872,13.526445
3,13.046217,12.922727,13.652875,12.198618,12.986252,13.249912,12.535716,11.141437,14.633,14.493407,...,13.333437,11.806583,12.820445,12.747840,12.862466,13.260457,13.557558,13.035242,13.418972,13.938128
4,12.350302,12.408989,13.256954,14.056298,13.762479,11.676288,12.533539,11.973285,14.633,14.586372,...,13.169335,12.753687,12.703800,12.256662,11.916656,13.308562,13.098402,13.059921,13.388373,13.328649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,13.083718,12.718774,13.792441,12.958023,13.613793,12.614123,12.365984,10.241127,14.633,14.545177,...,13.363698,14.068801,12.493127,12.679932,12.599035,12.915537,13.345572,13.016618,13.447438,13.969316
96,11.491588,11.701570,13.203345,13.007710,13.627666,12.007432,11.984633,12.894164,14.633,14.454095,...,13.392880,11.931445,12.736298,12.389966,12.332766,13.204210,13.436726,13.390301,13.371073,13.360163
97,12.585808,12.742912,13.844992,13.106648,13.718406,12.756096,13.442537,13.890015,14.633,14.676381,...,13.135813,14.172887,12.710286,12.166113,14.579181,12.991206,13.618024,13.078740,13.431747,13.831960
98,12.566109,12.542129,13.364735,13.730708,13.246534,12.818352,13.125130,12.269196,14.633,14.523864,...,13.315953,12.325198,13.118008,13.182427,12.730223,13.009928,13.547405,13.013507,13.418008,13.783058
